# Kuwala - Popularity Correlation

With this notebook you can correlate any value associated with a geo-reference with the Google popularity score. You
can upload your own file as a CSV. The only thing that is necessary to make it work is to have columns for latitude and
longitude and column headers.

The value columns can be specific to your use case, e.g., scooter bookings, sales in shops or crimes. The popularity
score is aggregated on a week. So ideally, the value columns that you want to correlate are aggregated on a weekly
timeframe as well.

As an example we are using an open data set from Uber that gives us the traversals of rides through specific hexagons.
You can find the raw data on their [open data platform](https://movement.uber.com/?lang=en-US). We preprocessed the raw
data so that the traversals are already aggregated per week.

### Kuwala

Kuwala is an open-source tool to build data products fast by integrating and connecting different data sources from
various domains. Other pipelines than the popularity score include worldwide scalable POI and demographics data. Check
out Kuwala on [GitHub](https://github.com/kuwala-io/kuwala) to access more data sources and functionalities.

## 1. Set Parameters

1. Set the file path to your CSV and the delimiter. Simply place your file under `notebooks/data` from within the
Jupyter environment or on your local file system.

In [ ]:
file_path = './data/lisbon_uber_traversals.csv'
delimiter = ';'

2. Set the H3 resolution to aggregate the results on. To see the average size of a hexagon at a given resolution go to
the [official H3 documentation](https://h3geo.org/docs/core-library/restable). The currently set resolution 8 has on
average an edge length of 0.46 km which can be freely interpreted as a radius. For this example we are using the
precalculated aggregations of the popularity score at resolution 8. To use a different resolution head over to
[Kuwala's repository](https://github.com/kuwala-io/kuwala) to set up your data warehouse.

In [ ]:
resolution = 8

3. Set the column names for the coordinates and the columns of the file you want to correlate.

In [ ]:
lat_column = 'latitude'
lng_column = 'longitude'
value_columns = ['weekly_traversals']

4. You can provide polygon coordinates as a GeoJSON-conforming array to select a subregion. Otherwise, data from the entire
country of Portugal will be analyzed. (The default coordinates are a rough representation of Lisbon, Portugal.)

In [ ]:
polygon_coords = [[[-9.092559814453125,38.794500078219826],[-9.164314270019531,38.793429729760994],[-9.217529296875,38.76666579487878],[-9.216842651367188,38.68792166352608],[-9.12139892578125,38.70399894245585],[-9.0911865234375,38.74551518488265],[-9.092559814453125,38.794500078219826]]]

## 2. Load dataframes

#### Load the file to correlate with the popularity score

In [ ]:
import h3
import pandas as pd
from geojson import Polygon


def add_h3_index_column(row):
    return h3.geo_to_h3(float(row[lat_column]), row[lng_column], resolution)


def polyfill_polygon(poly):
    h3_indexes = h3.polyfill(
        dict(poly),
        resolution,
        geo_json_conformant=True
    )

    return h3_indexes

df_file = pd.read_csv(file_path, sep=delimiter)
df_file['h3_index'] = df_file.apply(add_h3_index_column, axis=1)

if polygon_coords:
    polygon = Polygon(polygon_coords)
    h3_index_in_polygon = list(polyfill_polygon(poly=polygon))
    bool_series = df_file.h3_index.isin(h3_index_in_polygon)
    df_file = df_file[bool_series]

df_file = df_file[['h3_index', *value_columns]].groupby(['h3_index']).sum()

df_file.head(10)

#### Get weekly popularity per hexagon

In [ ]:
popularity = pd.read_csv('./data/portugal_popularity.csv', sep=';')

if polygon_coords:
    polygon = Polygon(polygon_coords)
    h3_index_in_polygon = list(polyfill_polygon(poly=polygon))
    bool_series = popularity.h3_index.isin(h3_index_in_polygon)
    popularity = popularity[bool_series]

popularity.head(10)

## 3. Join dataframes

In [ ]:
result = df_file.merge(popularity, on='h3_index', how='left')
result[['weekly_popularity']] = result[['weekly_popularity']].fillna(value=0)

result.head(10)

## 4. Visualize Results

#### Pandas Profiling Report

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(result, title="Pandas Profiling Report", explorative=True)

profile.to_notebook_iframe()

#### Map

In [ ]:
from unfolded.map_sdk import UnfoldedMap
from sidecar import Sidecar
from uuid import uuid4

unfolded_map = UnfoldedMap()
sc = Sidecar(title=f'Popularity Correlation', anchor='split-right')

with sc:
    display(unfolded_map)

dataset_id_combined=uuid4()

unfolded_map.add_dataset({
    'uuid': dataset_id_combined,
    'label': 'Correlated values',
    'data': result
})

unfolded_map.add_layer({
  'id': 'traversals',
  'type': 'hexagonId',
    'config': {
    'data_id': dataset_id_combined,
    "label": "Traversals",
    'columns': {
        'hex_id': 'h3_index'
    },
    'is_visible': True,
    'color_scale': 'quantile',
    'color_field': {
      'name': 'weekly_traversals',
      'type': 'real'
    }
  }
})